In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
import glob
import pandas as pd
import os
import seaborn as sns
import numpy as np
import torch.nn as nn

from tqdm import tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
# import pickle
import re
import json
from pathlib import Path

from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.examples.ubc_plotting_utils import *
from open_spiel.python.examples.ubc_cma import analyze_checkpoint

import dill as pickle
import types

from auctions.webutils import *

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

game = pyspiel.load_game('python_clock_auction', dict(filename='/apps/open_spiel/configs/small_sats_complete_symmetric.json'))


In [3]:
from open_spiel.python.examples.ubc_sample_game_tree import *

In [4]:
num_samples = 100
seed = 1234
history_prefix = [0, 0, 7, 7, 5, 5, 8, 4, 4]
player_0_checkpoint_pk=14437
player_1_checkpoint_pk=14437

checkpoint = EquilibriumSolverRunCheckpoint.objects.get(pk=player_0_checkpoint_pk)
# TODO: Modify this if you want to collect samples in parallel
env_params = EnvParams(track_stats=True, seed=seed, num_envs=1, sync=False, history_prefix=history_prefix)
env_and_policy = ppo_db_checkpoint_loader(checkpoint, env_params=env_params)

for player in range(2):
    checkpoint_pk = player_0_checkpoint_pk if player == 0 else player_1_checkpoint_pk
    checkpoint = EquilibriumSolverRunCheckpoint.objects.get(pk=checkpoint_pk)
    env_and_model_temp = ppo_db_checkpoint_loader(checkpoint)
    env_and_policy.agents[player] = env_and_model_temp.agents[player]

data = dict()


Using optimizer RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 1e-05
    foreach: None
    lr: 0.0003
    momentum: 0
    weight_decay: 0
)
Using optimizer RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 1e-05
    foreach: None
    lr: 0.0003
    momentum: 0
    weight_decay: 0
)
Using optimizer RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 1e-05
    foreach: None
    lr: 0.0003
    momentum: 0
    weight_decay: 0
)
Using optimizer RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 1e-05
    foreach: None
    lr: 0.0003
    momentum: 0
    weight_decay: 0
)
Using optimizer RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 1e-05
    foreach: None
    lr: 0.0003
    momentum: 0
    weight_decay: 0
)
Using optimizer RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 1e-05
    foreach: None
    lr: 0.0003
    momentum: 0
    weight_decay: 0
)


In [5]:

# Sample
actions = sample_game_from_position(env_and_policy, num_samples, seed=seed)
